<a href="https://colab.research.google.com/github/yscope75/CS2225.CH2001020/blob/master/Image_captioning_Master_courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.datasets as dset
import torchvision.datasets.utils as dset_utils
import torchvision.transforms as transforms
import os

In [2]:
import torch
from torch import nn
import torchvision
from torchsummary import summary

In [3]:
annotation_folder = os.path.join(os.path.abspath('.') + '/coco/annotations/')
image_folder = os.path.join(os.path.abspath('.') + '/coco/train2014/')
# Download and unzip annotations 
dset_utils.download_and_extract_archive(url='http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                        download_root=annotation_folder,
                                        extract_root=annotation_folder,
                                        filename='captions.zip')
# Download and unzion images
dset_utils.download_and_extract_archive(url='http://images.cocodataset.org/zips/train2014.zip',
                                        download_root=image_folder,
                                        extract_root=image_folder,
                                        filename='train2014.zip')

Extracting /content/coco/annotations/captions.zip to /content/coco/annotations/


Extracting /content/coco/train2014/train2014.zip to /content/coco/train2014/


In [4]:
# Setup device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [6]:
# Definition of main model
# Begin with Encoder
class Encoder(nn.Module):
  """
    Encode image input using pre-trained Resnet152 model on imagenet
  """
  def __init__(self, encoded_size=64):
    super(Encoder, self).__init__()
    resnet152 = torchvision.models.resnet152(pretrained=True)
    # remove the last two layers and keep the last CNN output 
    modules = list(resnet152.children())[:-2] 
    self.res_encoder = nn.Sequential(*modules) # last output (batch_size, 2048, 8, 8)
    # Flatten feature vector to (batch_size, 2048, 64)
    self.flat_embed = nn.Flatten(start_dim=2)
    # normalize variable input size to encoded size using adaptive pooling
    

  def forward(self, X_in):
    """
      The forward pass of encoder 
      args:
      - X_in: input data batch of size (batch, 3, Height, weight)
      return: encoded images of size (batch, embed_size, 64)
    """
    e_out = self.res_encoder(X_in) 
    # Flatten output vector to (batch_size, embedding_size, 64)
    e_out = self.flat_embed(e_out)
    # Change shape of output encoded to (batch_size, 64, embedding_dim)
    e_out = e_out.permute(0, 2, 1)
    return out
    
  

In [ ]:
class BahdanauAttention(nn.Module):
  """
    Define attention mechanism module on encoded image for genrating text
  """
  def __init__(self, encoder_dim, decoder_dim):
    """
      args: 
      - encoder_dim: size of encoded image
      - decoder_dim: size of encoder 
    """
    super(Attention, self).__init__()
    self.encoder_att = nn.Linear(encoder_dim, atten_dim)
    self.decoder_att = nn.Linear(decoder_dim, atten_dim)
    
    

In [ ]:
resent152 = torchvision.models.resnet152(pretrained=True)

In [ ]:
summary(resent152, input_size=(3,244,244))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 122, 122]           9,408
       BatchNorm2d-2         [-1, 64, 122, 122]             128
              ReLU-3         [-1, 64, 122, 122]               0
         MaxPool2d-4           [-1, 64, 61, 61]               0
            Conv2d-5           [-1, 64, 61, 61]           4,096
       BatchNorm2d-6           [-1, 64, 61, 61]             128
              ReLU-7           [-1, 64, 61, 61]               0
            Conv2d-8           [-1, 64, 61, 61]          36,864
       BatchNorm2d-9           [-1, 64, 61, 61]             128
             ReLU-10           [-1, 64, 61, 61]               0
           Conv2d-11          [-1, 256, 61, 61]          16,384
      BatchNorm2d-12          [-1, 256, 61, 61]             512
           Conv2d-13          [-1, 256, 61, 61]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
 t = torch.tensor([[[1, 2],
                       [3, 4]],
                      [[5, 6],
                       [7, 8]]], dtype=float)

In [ ]:
t.size()

torch.Size([2, 2, 2])

In [ ]:
a = nn.AdaptiveAvgPool2d(7)
a(t)

tensor([[[1.0000, 1.0000, 1.0000, 1.5000, 2.0000, 2.0000, 2.0000],
         [1.0000, 1.0000, 1.0000, 1.5000, 2.0000, 2.0000, 2.0000],
         [1.0000, 1.0000, 1.0000, 1.5000, 2.0000, 2.0000, 2.0000],
         [2.0000, 2.0000, 2.0000, 2.5000, 3.0000, 3.0000, 3.0000],
         [3.0000, 3.0000, 3.0000, 3.5000, 4.0000, 4.0000, 4.0000],
         [3.0000, 3.0000, 3.0000, 3.5000, 4.0000, 4.0000, 4.0000],
         [3.0000, 3.0000, 3.0000, 3.5000, 4.0000, 4.0000, 4.0000]],

        [[5.0000, 5.0000, 5.0000, 5.5000, 6.0000, 6.0000, 6.0000],
         [5.0000, 5.0000, 5.0000, 5.5000, 6.0000, 6.0000, 6.0000],
         [5.0000, 5.0000, 5.0000, 5.5000, 6.0000, 6.0000, 6.0000],
         [6.0000, 6.0000, 6.0000, 6.5000, 7.0000, 7.0000, 7.0000],
         [7.0000, 7.0000, 7.0000, 7.5000, 8.0000, 8.0000, 8.0000],
         [7.0000, 7.0000, 7.0000, 7.5000, 8.0000, 8.0000, 8.0000],
         [7.0000, 7.0000, 7.0000, 7.5000, 8.0000, 8.0000, 8.0000]]],
       dtype=torch.float64)